In [ ]:
import xarray as xr # allows working with netcdf files; mulitidimensional datasets with coordinates
import fsspec # allows me to open AWS S3 buckets (filesystem spec)
import numpy as np # allows matrix and array operations
import matplotlib.pyplot as plt
from dask.distributed import Client, progress # allows distributed computing (parallel conmputing)

In [ ]:
client = Client()
client

In [ ]:
%%time
# outputs the time it takes to run the cell (two percent signs followed by a command is called cell magic????)
# ds = xr.open_zarr(fsspec.get_mapper('s3://pangeo-data-uswest2/esip/NWM2/2017', anon=False, requester_pays=True))
# ds = xr.open_zarr(fsspec.get_mapper('s3://esip-qhub/noaa/nwm', anon=False, requester_pays=True))
ds = xr.open_zarr(fsspec.get_mapper('s3://noaa-nwm-retro-v2.0-zarr-pds/', anon=True), consolidated=True)

In [ ]:
ds # just calling the variable to print (if it wasn't the last thing in the current cell, use print ds)

In [ ]:
reaches = np.sort(np.array([10064725,10065359,10452338,10453362,10464069,10460567,8745093,1969311,11239079,11239377,10977177,25910528,8834960,8816997,10527139,9069044,25906434,9179824,9091348,9660512,20149927,20151001,9617395,9643525,20107091,9944828,9927684,14347320,6395537,18257633,16624798,16660513,21491880,79494,79564,79556,79468,16812521]))
print(reaches)

In [ ]:
ds['streamflow'].sel(feature_id=reaches) # or can do ds.streamflow and get the same thing; doing this without the .compute() shows what it will do without doing it

In [ ]:
%%time
dsflosup = ds.streamflow.sel(feature_id=reaches).compute() # .sel is select; must assign a variable name to actually get the output

In [ ]:
plt.figure(figsize=(10,8))
for reach in reaches: # this is like for reach=1:length(reaches)
    dsflosup.sel(feature_id=reach).plot(label=reach)
# plt.legend()
plt.show()

In [ ]:
dsflosup.to_netcdf('FloSupMajorStreams.nc')

In [ ]:
print(dsflosup.time[0], dsflosup.time[-1]) # to get just the values, could tack on .values after square brackets

In [ ]:
dsflosup.nbytes/1e6

In [ ]:
from bokeh.plotting import ColumnDataSource, figure, output_file, show

output_file("toolbar.html")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
))

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

In [ ]:
p